Polyhedral approximations with LazySets

In the [IntervalConstraintProgramming](/pages/tutorials/tutorialConstraintProgramming)
tutorial we saw how to solve the *set inversion* problem, i.e. how to find the region in
$\mathbb{R}^n$ that satisfies a set of constraints.

Interval constraint programming uses interval boxes. However, if the region we want to approximate
is not box-shaped, it will require a large number of boxes to be represented accurately.
To operate with those regions, it is practical to simplify such union of boxeswith another set representation,
hopefully without information loss, i.e. minimizing the *overapproximation error*.

The package \elink{LazySets.jl}{https://github.com/JuliaReach/LazySets.jl/} can be used to approximate regions
returned by `IntervalConstraintProgramming.jl` using general set types such as
\elink{polyhedra}{https://en.wikipedia.org/wiki/Polyhedron}, that is, finite intersections of half-spaces. More generally,
unions of polyhedra could be used.

First let us import the packages we need

In [ ]:
using IntervalArithmetic, IntervalConstraintProgramming, LazySets, Plots

The overapproximation of a paving using a polyhedron in constraint representation
(\elink{HPolyhedron}{https://juliareach.github.io/LazySets.jl/dev/lib/sets/HPolyhedron/}) with constraints in
the given directions `dirs` can be computed with the `overapproximate(::Paving, dirs::AbstractDirections)`
function as illustrated below.

In [ ]:
S = @constraint x^2+y^2 + 3*sin(x) + 5*sin(y) <= 1.0

X = IntervalBox(-10..10, 2) # our starting box
paving = pave(S, X, 0.02)

choose octagon directions

In [ ]:
Xoct = overapproximate(paving, OctDirections(2))

plot(Xoct, lab="Octagon", alpha=.5, c=:orange)
plot!(paving.inner, c="green", aspect_ratio=:equal, label="inner")
plot!(paving.boundary, c="gray", label="boundary")

The function `overapproximate` considers the union of the elements in the boundary of the paving
and then computes the support function of the such union along each chosen direction,
obtaining the corresponding polyhedron in constraint form.
In this example we have picked octagonal directions, but specifying custom directions is also possible.

It is interesting to note that the numer of boxes is much larger than the number of constraints of the approximation:

In [ ]:
length(paving.inner) + length(paving.boundary)

In [ ]:
length(Xoct.constraints)

When no directions are known a priori, we can also overapproximate using an iterative refinement process of
the given tolerance $\varepsilon$. First we construct $Y$, the (lazy, exact) convex hull of the paving's boundary,
then we overapproximate it using polygons:

In [ ]:
Y = ConvexHullArray(convert.(Hyperrectangle,paving.boundary))
Xpoly = overapproximate(Y, 0.1)
@show length(Xpoly.constraints)

Xpoly′ = overapproximate(Y, 0.01)
@show length(Xpoly′.constraints)

plot(Xoct, lab="Octagon", alpha=.5, c=:orange, legend=:bottomright)

plot!(Xpoly, lab="Polygon, ε=0.1")
plot!(Xpoly′, lab="Polygon, ε=0.01")

plot!(paving.boundary, lab="Paving (boundary)", c=:lightblue)
plot!(paving.inner, lab="Paving (inner)", c=:yellow)

lens!([0.0, 0.3], [0.0, 0.3], inset = (1, bbox(0.25, 0.35, 0.4, 0.4)))

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*